In [1]:
import os
import time
import pickle
import pdb

import numpy as np


import sklearn.preprocessing as skprep

import ot
import sklearn.metrics
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
from torch.utils.model_zoo import load_url
from torchvision import transforms

In [2]:
from cirtorch.networks.imageretrievalnet import init_network, extract_interest_points_mac
from cirtorch.datasets.datahelpers import cid2filename
from cirtorch.datasets.testdataset import configdataset
from cirtorch.utils.download import download_train, download_test
from cirtorch.utils.whiten import whitenlearn, whitenapply
from cirtorch.utils.evaluate import compute_map_and_print
from cirtorch.utils.general import get_data_root, htime

In [10]:
download_test(get_data_root())

>> Dataset oxford5k directory does not exist. Creating: /home/alexander/Documents/studies/20_ss/guided_research/code/ot4ir/data/test/oxford5k/jpg
>> Downloading dataset oxford5k archive oxbuild_images.tgz...
>> Extracting dataset oxford5k archive oxbuild_images.tgz...
>> Extracted, deleting dataset oxford5k archive oxbuild_images.tgz...
>> Downloading dataset oxford5k ground truth file...
>> Dataset paris6k directory does not exist. Creating: /home/alexander/Documents/studies/20_ss/guided_research/code/ot4ir/data/test/paris6k/jpg
>> Downloading dataset paris6k archive paris_1.tgz...
>> Extracting dataset paris6k archive paris_1.tgz...
>> Extracted, deleting dataset paris6k archive paris_1.tgz...
>> Downloading dataset paris6k archive paris_2.tgz...
>> Extracting dataset paris6k archive paris_2.tgz...
>> Extracted, deleting dataset paris6k archive paris_2.tgz...
>> Downloading dataset paris6k ground truth file...
>> Dataset roxford5k directory does not exist. Creating: /home/alexander/D

In [11]:
dataset = 'oxford5k'
network_path = 'http://cmp.felk.cvut.cz/cnnimageretrieval/data/networks/retrieval-SfM-120k/rSfM120k-tl-resnet50-gem-w-97bf910.pth'
image_size = 1024

In [12]:
state = load_url(network_path, model_dir=os.path.join(get_data_root(), 'networks'))
net_params = {}
net_params['architecture'] = state['meta']['architecture']
net_params['pooling'] = state['meta']['pooling']
net_params['local_whitening'] = state['meta'].get('local_whitening', False)
net_params['regional'] = state['meta'].get('regional', False)
net_params['whitening'] = state['meta'].get('whitening', False)
net_params['mean'] = state['meta']['mean']
net_params['std'] = state['meta']['std']
net_params['pretrained'] = False
net = init_network(net_params)
net.load_state_dict(state['state_dict'])
net.eval()
normalize = transforms.Normalize(
    mean=net.meta['mean'],
    std=net.meta['std']
)
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])
cfg = configdataset(dataset, os.path.join(get_data_root(), 'test'))

In [13]:
def plot_matches(ax, image1, image2, keypoints1, keypoints2, matches,
                 keypoints_color='k', matches_color=None, only_matches=False,
                 alignment='horizontal'):

    new_shape1 = list(image1.shape)
    new_shape2 = list(image2.shape)

    if image1.shape[0] < image2.shape[0]:
        new_shape1[0] = image2.shape[0]
    elif image1.shape[0] > image2.shape[0]:
        new_shape2[0] = image1.shape[0]

    if image1.shape[1] < image2.shape[1]:
        new_shape1[1] = image2.shape[1]
    elif image1.shape[1] > image2.shape[1]:
        new_shape2[1] = image1.shape[1]

    if new_shape1 != image1.shape:
        new_image1 = np.zeros(new_shape1, dtype=image1.dtype)
        new_image1[:image1.shape[0], :image1.shape[1]] = image1
        image1 = new_image1

    if new_shape2 != image2.shape:
        new_image2 = np.zeros(new_shape2, dtype=image2.dtype)
        new_image2[:image2.shape[0], :image2.shape[1]] = image2
        image2 = new_image2

    offset = np.array(image1.shape)
    if alignment == 'horizontal':
        image = np.concatenate([image1, image2], axis=1)
        offset[0] = 0
    elif alignment == 'vertical':
        image = np.concatenate([image1, image2], axis=0)
        offset[1] = 0
    else:
        mesg = ("plot_matches accepts either 'horizontal' or 'vertical' for "
                "alignment, but '{}' was given. See "
                "https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.plot_matches "  # noqa
                "for details.").format(alignment)
        raise ValueError(mesg)

    ax.imshow(image, interpolation='nearest', cmap='gray')
    ax.axis((0, image1.shape[1] + offset[1], image1.shape[0] + offset[0], 0))

    for i in range(matches.shape[0]):
        idx1 = matches[i, 0]
        idx2 = matches[i, 1]

        if matches_color is None:
            color = np.random.rand(3)
        else:
            color = matches_color

        ax.plot((keypoints1[idx1, 1], keypoints2[idx2, 1] + offset[1]),
                (keypoints1[idx1, 0], keypoints2[idx2, 0] + offset[0]),
                '-', color=color, linewidth=0.5)


In [14]:
## mutual matches ##
def mutual_max(P) :
    max0, max1 = P.max(1), P.max(0)
    indices0, indices1 = max0.indices, max1.indices
    mutual0 = torch.LongTensor(range(indices0.shape[0])) == indices1.gather(0, indices0)
    mutual1 = torch.LongTensor(range(indices1.shape[0])) == indices0.gather(0, indices1)
    zero = P.new_tensor(0)
    mscores0 = torch.where(mutual0, max0.values.exp(), zero)
    mscores1 = torch.where(mutual1, mscores0.gather(0, indices1), zero)

    valid0 = mutual0 & (mscores0 > 0)
    valid1 = mutual1 & valid0.gather(1, indices1)

    indices0 = torch.where(valid0, indices0, indices0.new_tensor(-1))
    indices1 = torch.where(valid1, indices1, indices1.new_tensor(-1))

    # make norm
    return indices0, indices1, mscores0, mscores1

In [15]:
q_idx = 0
p_idx = 0
n_idx = 0

qimage = [ cfg['qim_fname'](cfg, q_idx) ]
qgnd_images = [ cfg['im_fname'](cfg,j) for j in [p_idx] ]
qgndj_images = [ cfg['im_fname'](cfg,j) for j in [n_idx] ]

In [18]:
q_descriptors, q_locations, q_scores = extract_interest_points_mac(net, qimage, 1024, transform)
qgnd_descriptors, qgnd_locations, qgnd_scores = extract_interest_points_mac(net, qgnd_images, 1024, transform)
qgndj_descriptors, qgndj_locations, qgndj_scores = extract_interest_points_mac(net, qgndj_images, 1024, transform)


q_descriptors = q_descriptors.numpy()
q_locations = q_locations.numpy()

qgnd_descriptors = qgnd_descriptors.numpy()
qgnd_locations = qgnd_locations.numpy()

qgndj_descriptors = qgndj_descriptors.numpy()
qgndj_locations = qgndj_locations.numpy()

q_scores = q_scores.numpy()
qgnd_scores = qgnd_scores.numpy()
qgndj_scores = qgndj_scores.numpy()

## OT

select_1 = q_scores[0] > 25
scores_1 = q_scores[0, select_1]
desc_1 = q_descriptors[0, select_1, :]
loc_1 = q_locations[0, select_1, :]
loc_1, unique_1, _ = np.unique(loc_1, axis=0, return_index=True, return_inverse=True)
desc_1 = desc_1[unique_1, :]
scores_1 = scores_1[unique_1]

select_2 = qgnd_scores[j] > 10
scores_2 = qgnd_scores[j, select_2]
desc_2 = qgnd_descriptors[j, select_2, :]
loc_2 = qgnd_locations[j, select_2, :]
loc_2, unique_2, _ = np.unique(loc_2, axis=0, return_index=True, return_inverse=True)
desc_2 = desc_2[unique_2, :]
scores_2 = scores_2[unique_2]





NameError: name 'j' is not defined

In [ ]:
N_a = desc_1.shape[0]
N_b = desc_2.shape[0]

a = np.ones(N_a)
a = scores_1
a_sum = np.sum(a)

b = np.ones(N_b)
b = scores_2
b_sum = np.sum(b)

a = np.concatenate((a, np.array([b_sum])))
b = np.concatenate((b, np.array([a_sum])))

a = a / np.sum(a)
b = b / np.sum(b)

# do we need + 1?

P = ot.sinkhorn(a, b, M, 0.01, numItermax=2000)

ind_0, ind_1, _, _ = mutual_max(P)


In [ ]:
fig, ax = plt.subplots()
plot_matches(
    ax,
    img_1,
    img_2,
    loc_1,
    loc_2,
    max_match,
    matches_color='b')
ax.axis('off')
ax.set_title('OT correspondences')
plt.savefig('ot_' + str(loc_scale) + '_' + img_1_name + '_' + img_2_name + '_' + args.output_image, dpi=300)
plt.close()
